In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
from astropy.modeling.models import Sersic2D
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import detect_threshold, detect_sources
from astropy.stats import SigmaClip
from photutils.background import Background2D, MedianBackground
from astropy.io import fits
import time
import statmorph
%matplotlib inline

In [ ]:
from photutils.segmentation import make_2dgaussian_kernel
from astropy.convolution import convolve
kernel=make_2dgaussian_kernel(3.0,size=5)

In [ ]:
def obtain_clumpiness(name):
    
    #reads the data in the fits file
    path='/home/biju/statmorph_exercise/Conselice et al 2003 papers/FITS_file/'
    #nm=path+name+".fits"
    nm=path+name
    hdu=fits.open(nm)
    print("\nExecuting for ",name," : \n")
    data= hdu[0].data
    
    #calculate background
    sigma_clip=SigmaClip(sigma=3.0) #Remove data above 3-sigma level
    bkg_estimator= MedianBackground()
    bkg=Background2D(data,(200,200),filter_size=(3,3),
                     sigma_clip=sigma_clip,bkg_estimator=bkg_estimator)
    
    #background subtracted image
    final_data=data-bkg.background
    threshold=1.5*bkg.background_rms
    #convolved_data= convolve(final_data,kernel)
    
    #segmatation map
    seg_map=detect_sources(final_data,threshold,npixels=200)
    
    #as there are multiple sources but the galaxy concerned is the largest source, so choosing the largest segment
    t=seg_map.max_label
    max_area = 0
    max_index = None
    for i in range(1, t):
        area = seg_map.get_area(i)  # Assuming segmap.get_area(i) is your function
        if area > max_area:
            max_area = area
            max_index = i
    
    #running the statmorph code        
    start = time.time()
    source_morphs = statmorph.source_morphology(final_data, seg_map,gain=5)
    print('Time: %g s.' % (time.time() - start))
    
    #determine and extract the value for the galaxy
    morph = source_morphs[max_index]
    print(morph.smoothness)
    
    #val.append(morph.smoothness)
    #gal_name.append(name)